In [1]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

c:\ProgramData\Anaconda3\envs\llm_final\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
!nvcc --version
!pip show torch

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Jun_13_19:42:34_Pacific_Daylight_Time_2023
Cuda compilation tools, release 12.2, V12.2.91
Build cuda_12.2.r12.2/compiler.32965470_0
Name: torch
Version: 2.3.1
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: C:\ProgramData\Anaconda3\envs\llm_env\Lib\site-packages
Requires: filelock, fsspec, jinja2, mkl, networkx, sympy, typing-extensions
Required-by: accelerate, autoawq, autoawq_kernels, bitsandbytes, peft, sentence-transformers, torchaudio, torchvision


In [2]:
from transformers import pipeline

In [6]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline

In [2]:
from transformers import pipeline
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline

# 로컬 모델 경로 설정
local_model_path = "./llama-3.2-Korean-Bllossom-3B"

# HuggingFace pipeline 생성
hf_pipeline = pipeline(
    task="text-generation",
    model=local_model_path,
    config=f"{local_model_path}\\config.json",
    tokenizer=f"{local_model_path}\\tokenizer.json",
    device=0,  # GPU 사용, CPU를 사용하려면 -1로 설정
    trust_remote_code=True,
)

# HuggingFacePipeline 생성
llm = HuggingFacePipeline(
    pipeline=hf_pipeline,
    pipeline_kwargs=dict(
        max_new_tokens=512,
        do_sample=False,
        repetition_penalty=1.03,
        return_full_text=False,
    ),
)

# ChatHuggingFace 생성
chat_model = ChatHuggingFace(llm=llm)



RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
module 'torch.library' has no attribute 'register_fake'

In [ ]:
# ChatHuggingFace 사용 예제
response = chat_model.generate(["Hello! How are you?"])
print(response)

In [12]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "./EXAONE-3.5-2.4B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

AttributeError: partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "./models/EXAONE-3.5-2.4B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Choose your prompt
prompt = "Explain how wonderful you are"  # English example
prompt = "스스로를 자랑해 봐"       # Korean example
prompt = "철수가 20개의 연필을 가지고 있었는데 영희가 절반을 가져가고 민수가 남은 철수의 연필 중 5개를 가져갔으면 각각 몇개의 연필을 가지고있는가?"
messages = [
    {"role": "system", "content": "You are EXAONE model from LG AI Research, a helpful assistant."},
    {"role": "user", "content": prompt}
]
input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)

output = model.generate(
    input_ids.to("cuda"),
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=2048,
    do_sample=False,
)
print(tokenizer.decode(output[0]))

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.11s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


[|system|]You are EXAONE model from LG AI Research, a helpful assistant.[|endofturn|]
[|user|]철수가 20개의 연필을 가지고 있었는데 영희가 절반을 가져가고 민수가 남은 철수의 연필 중 5개를 가져갔으면 각각 몇개의 연필을 가지고있는가?
[|assistant|]철수의 초기 연필 개수는 20개였습니다.

1. **영희가 가져간 연필 수**: 철수의 연필의 절반이므로,
   \[
   \frac{20}{2} = 10 \text{개}
   \]
   영희는 철수로부터 10개의 연필을 가져갔습니다.

2. **철수가 남은 연필 수**: 초기 연필에서 영희가 가져간 연필을 뺀 값입니다.
   \[
   20 - 10 = 10 \text{개}
   \]
   철수는 남은 연필이 10개입니다.

3. **민수가 가져간 연필 수**: 철수가 남은 연필 중 5개를 가져갔으므로,
   \[
   10 - 5 = 5 \text{개}
   \]
   민수는 철수로부터 5개의 연필을 가져갔습니다.

따라서, 각 사람이 가지고 있는 연필 개수는 다음과 같습니다:
- 철수: **5개**
- 영희: **10개**
- 민수: **5개**[|endofturn|]
